In [2]:
import django_initializer
from backend.pymongo.mongodb import db
from datetime import datetime
import arrow

In [3]:
start_time = "2022-06-15"
end_time = "2022-06-15"
new_start_time = arrow.get(start_time).replace(hour=0, minute=0, second=0).datetime
display(new_start_time)
new_end_time = arrow.get(start_time).replace(hour=23, minute=59, second=59).datetime
display(new_end_time)

datetime.datetime(2022, 6, 15, 0, 0, tzinfo=tzutc())

datetime.datetime(2022, 6, 15, 23, 59, 59, tzinfo=tzutc())

# basic info

In [40]:
cursor=db.api_campaign.aggregate([
    {
        "$match":{
            "start_at":{
                "$gte": new_start_time
            },
            "end_at": {
                "$lte": new_end_time
            }
        }
    },
    {
        "$lookup": {
            "from": "api_campaign_comment",
            "as": "campaign_comment",
            'let': {'id': "$id" },
            "pipeline":[
                {"$match":{
                    '$expr': { '$eq': ["$$id", "$campaign_id"] },
                    "id":{"$ne":None}}
                }
            ]
        },
    },
    {
        "$lookup": {
            "from": "api_campaign_product",
            "as": "campaign_product_sold", 
            'let': {'id': "$id" },
            "pipeline":[
                {
                    "$match":{
                        '$expr': { '$eq': ["$$id", "$campaign_id"] },
                        "id":{"$ne":None},
                        "qty_sold": {
                            "$gt": 0
                        }
                    }
                }
            ]
        },
    },
    {
        "$lookup": {
            "from": "api_campaign_product",
            "as": "campaign_product_unsold", 
            'let': {'id': "$id" },
            "pipeline":[
                {
                    "$match":{
                        '$expr': { '$eq': ["$$id", "$campaign_id"] },
                        "id":{"$ne":None},
                        "qty_sold": {
                            "$eq": 0
                        }
                    }
                }
            ]
        },
    },
    {
        "$lookup": {
            "from": "api_campaign_product",
            "as": "campaign_product_total_item", 
            'let': {'id': "$id" },
            "pipeline":[
                {
                    "$match":{
                        '$expr': { '$eq': ["$$id", "$campaign_id"] },
                        "id":{"$ne":None},
                    }
                }
            ]
        },
    },
    {
        "$lookup": {
            "from": "api_order",
            "as": "orders", 
            'let': {'id': "$id" },
            "pipeline":[
                {
                    "$match":{
                        '$expr': { '$eq': ["$$id", "$campaign_id"] },
                        "id":{"$ne":None},
                    }
                },
                {
                    "$project":{
                        "_id":0,
                        "count":{"$sum":1},
                        "sum":{"$sum":"$total"},

                    }
                }
            ]
        },
    },
    {
        "$lookup": {
            "from": "api_pre_order",
            "as": "pre_orders", 
            'let': {'id': "$id" },
            "pipeline":[
                {
                    "$match":{
                        '$expr': { '$eq': ["$$id", "$campaign_id"] },
                        "id":{"$ne":None},
                    }
                },
                {
                    "$project":{
                        "_id":0,
                        "count":{"$sum":1},
                        "sum":{"$sum":"$total"},

                    }
                }
            ]
        },
    },
    {
        "$project":{
            "_id":0,
            "campaign_id":"$id",
            "post_comment":{"$size":"$campaign_comment.id"},
            "no_of_items_sold":{"$size": "$campaign_product_sold.id"},
            "no_of_items_unsold":{"$size": "$campaign_product_unsold.id"},
            "total_no_of_items":{"$size": "$campaign_product_total_item.id"},
            "total_inventories":{"$sum": "$campaign_product_total_item.qty_for_sale"},
            
            "total_no_of_orders": {"$sum":{"$add":["$pre_orders.count","$orders.count"]}},
            "total_amount": {"$sum":{"$add":["$pre_orders.sum","$orders.sum"]}},
            
#             "average_order_value":{"$avg": "$orders.total"}
            
        }
    }
])
basic_info = list(cursor)

OperationFailure: PlanExecutor error during aggregation :: caused by :: $add only supports numeric or date types, not array, full error: {'ok': 0.0, 'errmsg': 'PlanExecutor error during aggregation :: caused by :: $add only supports numeric or date types, not array', 'code': 16554, 'codeName': 'Location16554', '$clusterTime': {'clusterTime': Timestamp(1657534514, 2), 'signature': {'hash': b'\xef\x8e\x81P<0D%\xbaZ\x8b\xe3\x85\xcf\xca\x94\xe7|\xf1\x1e', 'keyId': 7061090302374707201}}, 'operationTime': Timestamp(1657534514, 2)}

In [41]:
basic_info

[{'campaign_id': 495,
  'post_comment': 2,
  'no_of_items_sold': 1,
  'no_of_items_unsold': 2,
  'total_no_of_items': 3,
  'total_inventories': 1000,
  'average_order_value': None,
  'total_amount': 0,
  'total_no_of_orders': 0},
 {'campaign_id': 496,
  'post_comment': 4,
  'no_of_items_sold': 4,
  'no_of_items_unsold': 2,
  'total_no_of_items': 6,
  'total_inventories': 1050,
  'average_order_value': 127.92,
  'total_amount': 127.92,
  'total_no_of_orders': 1},
 {'campaign_id': 497,
  'post_comment': 14,
  'no_of_items_sold': 1,
  'no_of_items_unsold': 1,
  'total_no_of_items': 2,
  'total_inventories': 115,
  'average_order_value': 12.5,
  'total_amount': 25.0,
  'total_no_of_orders': 2},
 {'campaign_id': 499,
  'post_comment': 13,
  'no_of_items_sold': 3,
  'no_of_items_unsold': 1,
  'total_no_of_items': 4,
  'total_inventories': 400,
  'average_order_value': 8870.0,
  'total_amount': 8870.0,
  'total_no_of_orders': 1},
 {'campaign_id': 501,
  'post_comment': 16,
  'no_of_items_sold

# order analysis

In [11]:
cursor=db.api_campaign.aggregate([
    {
        "$match":{
            "start_at":{
                "$gte": new_start_time
            },
            "end_at": {
                "$lte": new_end_time
            }
        }
    },
    {
        "$lookup": {
            "from": "api_order",
            "as": "orders", 
            'let': {'id': "$id" },
            "pipeline":[
                {
                    "$match":{
                        '$expr': { '$eq': ["$$id", "$campaign_id"] },
                        "id":{"$ne":None},
                    }
                },
                { "$group": {
                    "_id": "$status",
                    "qty": {"$sum": 1}
                }}
            ]
        },
    },
    {
        "$project":{
            "_id":0,
            "campaign_id":"$id",
            "status": "$orders._id",
            "qty": "$orders.qty"
            
        }
    }
])
order = list(cursor)

In [12]:
order

[{'campaign_id': 495, 'status': [], 'qty': []},
 {'campaign_id': 496, 'status': ['complete'], 'qty': [1]},
 {'campaign_id': 497, 'status': ['complete'], 'qty': [2]},
 {'campaign_id': 499, 'status': ['complete'], 'qty': [1]},
 {'campaign_id': 501, 'status': ['complete'], 'qty': [1]},
 {'campaign_id': 502, 'status': [], 'qty': []}]

In [13]:
cursor=db.api_campaign.aggregate([
    {
        "$match":{
            "start_at":{
                "$gte": new_start_time
            },
            "end_at": {
                "$lte": new_end_time
            }
        }
    },
    {
        "$lookup": {
            "from": "api_pre_order",
            "as": "pre_orders", 
            'let': {'id': "$id" },
            "pipeline":[
                {
                    "$match":{
                        '$expr': { '$eq': ["$$id", "$campaign_id"] },
                        "id":{"$ne":None},
                    }
                },
                { "$group": {
                    "_id": "$status",
                    "qty": {"$sum": 1}
                }}
            ]
        },
    },
    {
        "$project":{
            "_id":0,
            "campaign_id":"$id",
            "status": "$pre_orders._id",
            "qty": "$pre_orders.qty"
            
        }
    }
])
order = list(cursor)
order

[{'campaign_id': 495, 'status': ['review'], 'qty': [1]},
 {'campaign_id': 496, 'status': ['review'], 'qty': [2]},
 {'campaign_id': 497, 'status': ['review'], 'qty': [2]},
 {'campaign_id': 499, 'status': ['review'], 'qty': [1]},
 {'campaign_id': 501, 'status': ['review'], 'qty': [3]},
 {'campaign_id': 502, 'status': ['review'], 'qty': [2]}]

In [39]:
cursor=db.api_campaign.aggregate([
    {
        "$match":{
            "start_at":{
                "$gte": new_start_time
            },
            "end_at": {
                "$lte": new_end_time
            }
        }
    },
    {
        "$lookup": {
            "from": "api_campaign_product",
            "as": "campaign_product", 
            'let': {'id': "$id" },
            "pipeline":[
                {
                    "$match":{
                        '$expr': { '$eq': ["$$id", "$campaign_id"] },
                        "id":{"$ne":None},
                    }
                },
                {
                    "$lookup": {
                        "from": "api_order_product",
                        "as": "order_product", 
                        'let': {'id': "$id" },
                        "pipeline":[
                            {
                                "$match":{
                                    
                                    '$expr': {"$eq": ["$$id", "$campaign_product_id"]},
                                    "id":{"$ne":None},
                                }
                            },
                            {
                                "$project":{
                                    "_id":0,
                                    "qty":1,
                                }
                            }
                        ]
                    },
                },
                {
                    "$project":{
                        "_id":0,
                        "name":1,
                        "order_product_qty":{"$sum":"$order_product.qty"},
                        "qty_for_sale":1,
                        "status": {
                            "$switch": {
                              "branches": [
                                {
                                  "case": {"$eq": ["$qty_for_sale", "$order_product_qty"] },
                                  "then": "Sold Out"
                                }
                              ],
                              "default": ""
                            }

                        }
                    }
                },
                {"$sort": {"order_product_qty":-1}},
                { "$limit": 10 }
            ]
        },
    },
    {
        "$project":{
            "_id":0,
            "campaign_id":"$id",
            "Best selling items-Top 10": "$campaign_product.name",
            "qty_for_sale": "$campaign_product.qty_for_sale",
            "order_product_qty": "$campaign_product.order_product_qty",
            "status": "$campaign_product.status"
            
        }
    },
    
])
order = list(cursor)
order


[{'campaign_id': 495,
  'Best selling items-Top 10': ['Top', 'Jeans', 'Skirt'],
  'qty_for_sale': [200, 500, 300],
  'order_product_qty': [1, 0, 0],
  'status': ['', '', '']},
 {'campaign_id': 496,
  'Best selling items-Top 10': ['grapegfruit',
   'Pepsi',
   'Skirt',
   'Thai Meow Tea',
   'prize yummy cookies',
   'Prize - Bag'],
  'qty_for_sale': [200, 300, 400, 100, 40, 10],
  'order_product_qty': [6, 2, 2, 1, 0, 0],
  'status': ['', '', '', '', '', '']},
 {'campaign_id': 497,
  'Best selling items-Top 10': ['nina0615', 'nina0615'],
  'qty_for_sale': [100, 15],
  'order_product_qty': [3, 0],
  'status': ['', '']},
 {'campaign_id': 499,
  'Best selling items-Top 10': ['cc013',
   'cecitest12',
   'cecitest5gift',
   'cecitest6'],
  'qty_for_sale': [100, 100, 100, 100],
  'order_product_qty': [10, 3, 1, 0],
  'status': ['', '', '', '']},
 {'campaign_id': 501,
  'Best selling items-Top 10': ['Lynn_0615', '0608lynnlucky draw', 'll615_2'],
  'qty_for_sale': [50, 3, 5],
  'order_product_